In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pylab as plt

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
# 加载数据
X = pd.read_pickle('./data1/X_train.pkl')
y = pd.read_pickle('./data1/y_train.pkl')

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4254292 entries, 0 to 1063572
Data columns (total 35 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   14_Sum_order_detail_amount     float16
 1   14_Max_order_detail_amount     float16
 2   14_Sum_order_detail_goods_num  float16
 3   14_Max_order_detail_goods_num  float16
 4   14_Sum_order_detail_payment    float16
 5   14_Max_order_detail_payment    float16
 6   14_Count_order                 uint16 
 7   30_Sum_order_detail_amount     float16
 8   30_Max_order_detail_amount     float16
 9   30_Sum_order_detail_goods_num  float16
 10  30_Max_order_detail_goods_num  float16
 11  30_Sum_order_detail_payment    float16
 12  30_Max_order_detail_payment    float16
 13  30_Count_order                 uint16 
 14  60_Sum_order_detail_amount     float16
 15  60_Max_order_detail_amount     float16
 16  60_Sum_order_detail_goods_num  float16
 17  60_Max_order_detail_goods_num  float16
 18  60

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [14]:
model = xgb.XGBClassifier(max_depth=6,learning_rate= 0.1, verbosity=1, n_estimators=100,
                          objective='binary:logistic', eval_metric='auc', scale_pos_weight=1)

In [15]:
model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="auc",
        eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.59925
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.59932
[2]	validation_0-auc:0.59933
[3]	validation_0-auc:0.59949
[4]	validation_0-auc:0.59952
[5]	validation_0-auc:0.59949
[6]	validation_0-auc:0.59959
[7]	validation_0-auc:0.59960
[8]	validation_0-auc:0.59964
[9]	validation_0-auc:0.59965
[10]	validation_0-auc:0.59965
[11]	validation_0-auc:0.59967
[12]	validation_0-auc:0.59972
[13]	validation_0-auc:0.59969
[14]	validation_0-auc:0.59973
[15]	validation_0-auc:0.59972
[16]	validation_0-auc:0.59973
[17]	validation_0-auc:0.59977
[18]	validation_0-auc:0.59975
[19]	validation_0-auc:0.59985
[20]	validation_0-auc:0.59983
[21]	validation_0-auc:0.59987
[22]	validation_0-auc:0.59993
[23]	validation_0-auc:0.59989
[24]	validation_0-auc:0.60002
[25]	validation_0-auc:0.60000
[26]	validation_0-auc:0.60008
[27]	validation_0-auc:0.60007
[28]	validation_0-auc:0.60010
[29]	validation_0-auc:0.60020
[30]	validation_0-auc:0.60018
[31]	validation_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=1)

In [8]:
X_predict = pd.read_pickle('./data1/X_test.pkl')
y_predict = pd.read_pickle('./data1/y_test.pkl')
y_predict = pd.DataFrame(y_predict)

In [16]:
y_predict['result'] = model.predict(X_predict)

In [19]:
result = pd.read_csv('./data/submission.csv')
tmp = pd.merge(result[['customer_id']], y_predict[['customer_id','result']], on='customer_id', how='left')
tmp['result'].fillna(0, inplace=True)

In [22]:
result['result'] = tmp['result']

In [23]:
result.to_csv('submission_xgb.csv', index = False)